In [234]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import dash_table
import dash
import pandas as pd
import json
import import_ipynb
import functions
import charts
import tables
import csv

In [235]:
# Build App
app = JupyterDash(__name__,suppress_callback_exceptions=True)
#function to query dbpedia endpoint
def sql(value):
#     sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(value)
    sparql.setReturnFormat(JSON)
    result = sparql.query()
    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)
app.layout = html.Div([
    
    html.Div(
        className="navbar",
        children=[ 
            html.H3(
                children="Interactive Data Analytics Dashboard for RDF Knowledge Graphs", 
                className="navbar--title"),
            html.Div(
                children=[
                dcc.Tabs(id='tabs', value='Dashboard', children=[
                    dcc.Tab(label='Dashboard', value='Dashboard'),
                    dcc.Tab(label='Tables', value='Tables'),
                    dcc.Tab(label='Charts', value='Charts'),
                    dcc.Tab(label='Query Samples', value='Samples'),
                    dcc.Tab(label='Upload', value='Upload'),
                    dcc.Tab(label='Download', value='Download'),
                ]),
                html.Div(id='tabs-content', className="tabs-content")
            ])
    ]),
    html.Div(
        className="querybox",
        children=[
            html.H4(
                className="querybox--title",
                children="SPARQL Query"
            ),
            dcc.Textarea(
                id= "query-endpoint", 
                value='https://query.wikidata.org/sparql',
                placeholder="Enter your SPARQL query endpoint.",
                className="querybox--endpoint"
            ),
            dcc.Textarea(
                id= "query-text", 
                value="",
                placeholder="Enter your SPARQL query.",
                className="querybox--textarea"
            ),
            html.Button(
                id="submit-btn", className="querybox--btn", 
                children="SUBMIT", 
                n_clicks=0
            )
    ]),      
])#layout div

with open('Book2.csv', 'rt') as f:
    csv_reader = csv.reader(f)
    headers = []
    queries = []
    for line in csv_reader:
        headers.append(line[0])
        queries.append(line[1])

    data = {'Query Examples':  headers}
    df = pd.DataFrame (data, columns = ['Query Examples'])
    ResultListdataframe = pd.DataFrame (data=[], columns = [])

#--------------------------------------------------------------------------------------------------------------------
#initial empty table
start_table_df = pd.DataFrame(columns=[''])


#render page for selected tab
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value'),
)
def render_content(tab):
    if tab == 'Dashboard':
        return html.Div([
        ])
    elif tab == 'Tables':
        return html.Div([
            html.H3('Tables'),
            html.Div(
                id='MainTableDiv',
                children=[
                    dash_table.DataTable(
                        id='Resulttable',
                        data=start_table_df.to_dict('records'), 
                        columns = [{'id': c, 'name': c} for c in start_table_df.columns],
                        style_cell=dict(textAlign='left'),   
                        editable=True,
                        filter_action="native",
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable="single",
                        row_selectable="multi",
                        row_deletable=True,
                        selected_columns=[],
                        selected_rows=[],
                        page_action="native",
                        page_current= 0,
                        page_size= 10,
                        export_format="csv"
                    )
                ],
                style= {'display': 'none'}
            ),
        ])
    elif tab == 'Charts':
        return html.Div([
            html.H3('Charts'),
            html.Div(
                id="result-graph",
                className="result-graph",
                children=[
                    dcc.Graph(id='graph')
                ],
                style= {'display': 'none'}
            )
        ])
    elif tab == 'Samples':
        return html.Div([
            html.H3('Query Samples'),
            html.Div(
                className="examples",
                children=[
                    dash_table.DataTable(
                        id='querySamples',
                        columns=[{"name": i, "id": i} for i in df.columns],
                        data=df.to_dict('records'),
                        style_table={'width': '800px'},
                        style_cell=dict(textAlign='left'), 
                        column_selectable='single'
                    )
                ]),
        ])
@app.callback(
    Output('query-text', 'value'),
    Input("querySamples", "active_cell")
)
def get_active_cell(active_cell):
#     print(next(iter(active_cell.values())))
#     print(queries[0])
    return queries[next(iter(active_cell.values()))]
    
@app.callback(
    [Output("Resulttable", "data"), Output('Resulttable', 'columns'), Output('MainTableDiv', 'style')],
    [Input("submit-btn", "n_clicks"),
    Input("intermediate-value", "children"),],
    #[State("query-text","value")]
)    
#def gen_table(n_clicks,page_current,page_size, value):
def gen_table(n_clicks, jsonified_ResultListdataframe):

    if n_clicks > 0:
        #resultList = sql(value)
        #ResultListdataframe = resultList
        resultListdataframe = pd.read_json(jsonified_ResultListdataframe, orient='split')
        mycolumns = [{'name': index, 'id': index,"deletable": True, "selectable": True} for index in resultListdataframe.columns]
#         print(mycolumns)   
        return resultListdataframe.to_dict('records'),mycolumns,{'display': 'block'}
    return start_table_df.to_dict('records'), [{'id': '', 'name': ''}],{'display': 'none'}


@app.callback(
    [Output("graph", "figure"), Output('result-graph', 'style')],
    Input("submit-btn", "n_clicks"),
    State("query-text","value")
)
def gen_graph(n_clicks, value):
    fig = html.Div()
    if n_clicks > 0:
        resultList = sql(value)
        ResultListdataframe = resultList
#         print(ResultListdataframe)
        fig = px.scatter(ResultListdataframe)
#         fig = px.bar(ResultListdataframe)
#         fig = px.line(ResultListdataframe)
    
    return fig,{'display': 'block'}


In [35]:
# Run app and display result inline in the notebook
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8080/
                                            item           itemLabel  \
0          http://www.wikidata.org/entity/Q22686        Donald Trump   
1           http://www.wikidata.org/entity/Q4617         Corbin Bleu   
2          http://www.wikidata.org/entity/Q19848           Lady Gaga   
3          http://www.wikidata.org/entity/Q33866  Theodore Roosevelt   
4          http://www.wikidata.org/entity/Q25089         Woody Allen   
...                                          ...                 ...   
28966  http://www.wikidata.org/entity/Q104369165            Jack Val   
28967  http://www.wikidata.org/entity/Q104369390          Jimmy Dale   
28968  http://www.wikidata.org/entity/Q104439551       Gloria Walker   
28969  http://www.wikidata.org/entity/Q104589883      Kamani Batista   
28970  http://www.wikidata.org/entity/Q104673574    Bertram H. Lubin   

                                         itemDescription sitelinks  
0        45th and curre